<a href="https://colab.research.google.com/github/midhushi-m/Comic_Generation_Hindi/blob/main/comic_strip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

# Change directory to your model folder
import os
os.chdir("/content/drive/My Drive/character")


Mounted at /content/drive


In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


CUDA Available: True
CUDA Device Name: Tesla T4


In [ ]:
'''import json
import re

def extract_characters(text):
    # Simple pattern to catch names before verbs like 'कहता है', 'कहती है'
    name_pattern = r"(\w+)\s+(कहता है|कहती है|बोला|बोली)"
    characters = set()

    # Find all names before dialogue markers
    for match in re.finditer(name_pattern, text):
        characters.add(match.group(1))

    # Additional manual patterns for this specific story
    if "राम" in text:
        characters.add("राम")
    if "कालू" in text:
        characters.add("कालू")
    if "सोनी" in text:
        characters.add("सोनी")

    return list(characters)

def split_scenes(text):
    sentences = [s.strip() + "।" for s in text.split("।") if s.strip()]
    scenes = []
    all_characters = extract_characters(text)
    current_location = "अज्ञात"

    for i, sentence in enumerate(sentences, 1):
        # Update location
        if "जंगल" in sentence:
            current_location = "जंगल"
        elif "झील" in sentence:
            current_location = "झील"

        # Characters in scene
        chars_in_scene = [c for c in all_characters if c in sentence]
        if "तीनों" in sentence:
            chars_in_scene = all_characters

        # Detect dialogue between single quotes
        dialogue = None
        if "'" in sentence:
            parts = sentence.split("'")
            if len(parts) >= 3:
                dialogue_text = parts[1].strip("।")
                speaker_match = re.search(r"(\w+)\s+(?:कहता है|कहती है|बोलता है|बोलती है)", sentence)
                speaker = speaker_match.group(1) if speaker_match else "unknown"
                dialogue = {
                    "character": speaker,
                    "text": dialogue_text,
                    "emotion": detect_emotion(dialogue_text),
                    "position": detect_position(sentence, speaker)
                }


        scene_data = {
            "scene_number": i,
            "location": current_location,
            "scene_text": sentence,
            "characters_in_scene": chars_in_scene,
            "emotion": detect_scene_emotion(sentence),
            "position": detect_scene_position(sentence),
            "dialogues": [dialogue] if dialogue else []
        }

        scenes.append(scene_data)

    return scenes

# Helper functions for emotion and position detection
def detect_emotion(text):
    text = text.lower()
    if any(word in text for word in ["खुश", "हँस", "मुस्कुर"]):
        return "happy"
    elif any(word in text for word in ["डर", "भय"]):
        return "scared"
    elif any(word in text for word in ["दुख", "उदास"]):
        return "sad"
    return "smile"  # Default

def detect_position(sentence, character):
    sentence = sentence.lower()
    if "बैठ" in sentence:
        return "sitting"
    elif "लेट" in sentence:
        return "lying"
    elif "चल" in sentence:
        return "walking"
    return "standing"  # Default

def detect_scene_emotion(sentence):
    # Similar logic as detect_emotion but for whole scene
    return detect_emotion(sentence)

def detect_scene_position(sentence):
    # Similar logic as detect_position but for whole scene
    if "बैठ" in sentence:
        return "sitting"
    return "standing"

# Hindi story
hindi_story = """
राम, एक सुंदर लड़का, जंगल में अपने बात करने वाले कुत्ते कालू के साथ चल रहा है।
वहाँ उसकी मुलाकात सोनी नाम की एक लड़की से होती है। कालू कहता है 'नमस्ते'। सोनी कहती है 'हाय'।
बाद में तीनों एक झील के पास बैठते हैं और बातें करते हैं।
"""

# Process the story
characters = extract_characters(hindi_story)
scenes = split_scenes(hindi_story)

story_data = {
    "original_hindi": hindi_story.strip(),
    "characters": characters,
    "scenes": scenes
}

# Save to JSON
with open("hindi_story_processed.json", "w", encoding="utf-8") as f:
    json.dump(story_data, f, ensure_ascii=False, indent=4)

print("✅ Hindi story processed successfully!")'''

✅ Hindi story processed successfully!


In [ ]:
import json
import re

def extract_characters(text):
    name_pattern = r"(\w+)\s+(कहता है|कहती है|बोला|बोली|बोलता है|बोलती है)"
    characters = set()

    for match in re.finditer(name_pattern, text):
        characters.add(match.group(1))

    # Additional manual patterns for this specific story
    for name in ["राम", "कालू", "सोनी"]:
        if name in text:
            characters.add(name)

    return list(characters)

def split_scenes(text):
    sentences = [s.strip() + "।" for s in text.split("।") if s.strip()]
    scenes = []
    all_characters = extract_characters(text)
    current_location = "अज्ञात"

    for i, sentence in enumerate(sentences, 1):
        if "जंगल" in sentence:
            current_location = "जंगल"
        elif "झील" in sentence:
            current_location = "झील"

        chars_in_scene = [c for c in all_characters if c in sentence]
        if "तीनों" in sentence or "सभी" in sentence:
            chars_in_scene = all_characters

        # Detect dialogue text inside single quotes
        dialogues = []
        matches = re.findall(r"'(.*?)'", sentence)
        for d in matches:
            dialogues.append(d.strip("।"))

        scene_data = {
            "scene_number": i,
            "location": current_location,
            "scene_text": sentence,
            "characters_in_scene": chars_in_scene,
            "emotion": detect_scene_emotion(sentence),
            "position": detect_scene_position(sentence),
            "dialogues": dialogues
        }

        scenes.append(scene_data)

    return scenes

def detect_emotion(text):
    text = text.lower()
    if any(word in text for word in ["खुश", "हँस", "मुस्कुर"]):
        return "happy"
    elif any(word in text for word in ["डर", "भय"]):
        return "scared"
    elif any(word in text for word in ["दुख", "उदास"]):
        return "sad"
    return "smile"

def detect_position(sentence, character):
    sentence = sentence.lower()
    if "बैठ" in sentence:
        return "sitting"
    elif "लेट" in sentence:
        return "lying"
    elif "चल" in sentence:
        return "walking"
    return "standing"

def detect_scene_emotion(sentence):
    return detect_emotion(sentence)

def detect_scene_position(sentence):
    if "बैठ" in sentence:
        return "sitting"
    return "standing"

# Hindi story
hindi_story = """
राम, एक सुंदर लड़का, जंगल में अपने बात करने वाले कुत्ते कालू के साथ चल रहा है।
वहाँ उसकी मुलाकात सोनी नाम की एक लड़की से होती है। कालू कहता है 'नमस्ते'। सोनी कहती है 'हाय'।
बाद में तीनों एक झील के पास बैठते हैं और बातें करते हैं।
"""

characters = extract_characters(hindi_story)
scenes = split_scenes(hindi_story)

story_data = {
    "original_hindi": hindi_story.strip(),
    "characters": characters,
    "scenes": scenes
}

with open("hindi_story.json", "w", encoding="utf-8") as f:
    json.dump(story_data, f, ensure_ascii=False, indent=4)

print("✅ Hindi story processed successfully!")


✅ Hindi story processed successfully!


In [ ]:
!pip install diffusers transformers accelerate bitsandbytes xformers safetensors
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
import os
import torch
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import functional as F
from diffusers import StableDiffusionPipeline, DDPMScheduler
from peft import LoraConfig, get_peft_model
from google.colab import drive

drive.mount('/content/drive')

# ✅ Set Random Seeds for Reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# ✅ Configuration
class Config:
    model_id = "stabilityai/stable-diffusion-2-1-base"
    dataset_dir = "/content/drive/MyDrive/character/train"
    output_dir = "/content/drive/MyDrive/lora_trained_model"
    batch_size = 1
    gradient_accumulation_steps = 4
    learning_rate = 1e-5
    num_epochs = 10
    mixed_precision = "fp16"
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.1
    target_modules = ["to_q", "to_k", "to_v", "to_out.0"]
    validation_steps = 100
    characters = sorted(os.listdir(dataset_dir))  # Sort characters for consistent order

config = Config()

# ✅ Load Stable Diffusion Model
pipe = StableDiffusionPipeline.from_pretrained(
    config.model_id, torch_dtype=torch.float16 if config.mixed_precision == "fp16" else torch.float32
)
pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# ✅ Apply LoRA
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=config.target_modules,
    lora_dropout=config.lora_dropout
)
pipe.unet = get_peft_model(pipe.unet, lora_config)
pipe.unet.train()

# ✅ Define Data Augmentation & Custom Image Dataset
class ImageDataset(Dataset):
    def __init__(self, root_dir, character, transform=None):
        self.root_dir = os.path.join(root_dir, character)
        self.character = character
        self.transform = transform
        self.image_files = []
        self.labels = []

        for emotion in sorted(os.listdir(self.root_dir)):
            emotion_path = os.path.join(self.root_dir, emotion)
            if not os.path.isdir(emotion_path):  # ✅ Skip files
                continue

            emotion_path = os.path.join(self.root_dir, emotion)
            if not os.path.isdir(emotion_path):
                continue

            for file in os.listdir(emotion_path):
                if file.endswith((".png", ".jpg", ".jpeg")):
                    self.image_files.append(os.path.join(emotion_path, file))
                    self.labels.append(f"{character}_{emotion}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        prompt = f"A {label.split('_')[1]} {label.split('_')[0]} in comic style"
        if self.transform:
            image = self.transform(image)
        return {"pixel_values": image, "prompt": prompt}

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# ✅ Training Loop for Each Character
for character in config.characters:
    print(f"\n🚀 Training for character: {character}")
    char_dataset = ImageDataset(config.dataset_dir, character, transform=transform)
    char_dataloader = DataLoader(char_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2, pin_memory=True)

    optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=config.learning_rate, weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(char_dataloader) * config.num_epochs // config.gradient_accumulation_steps)

    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(char_dataloader), desc=f"{character} - Epoch {epoch+1}")
        optimizer.zero_grad()

        for step, batch in enumerate(char_dataloader):
            pixel_values = batch["pixel_values"].to("cuda").half()
            latents = pipe.vae.encode(pixel_values).latent_dist.sample() * pipe.vae.config.scaling_factor
            timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (latents.shape[0],), device="cuda").long()
            noise = torch.randn_like(latents)
            noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)
            text_input = pipe.tokenizer(batch["prompt"], padding="max_length", max_length=pipe.tokenizer.model_max_length, return_tensors="pt").to("cuda")
            encoder_hidden_states = pipe.text_encoder(**text_input).last_hidden_state
            model_pred = pipe.unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states).sample
            loss = F.mse_loss(model_pred, noise) / config.gradient_accumulation_steps
            loss.backward()

            if (step + 1) % config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(pipe.unet.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        progress_bar.close()

    # ✅ Save Checkpoint for Each Character
    char_save_dir = os.path.join(config.output_dir, f"{character}_lora")
    os.makedirs(char_save_dir, exist_ok=True)
    lora_save_dir = os.path.join(char_save_dir, "lora_adapter")
    pipe.unet.save_pretrained(lora_save_dir)
    print(f"✅ LoRA model saved for {character} at {char_save_dir}")

print("🎉 Training Completed for All Characters!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]


🚀 Training for character: Bruno


Bruno - Epoch 10:   0%|          | 0/29 [00:15<?, ?it/s]


✅ LoRA model saved for Bruno at /content/drive/MyDrive/lora_trained_model/Bruno_lora

🚀 Training for character: Ram


Ram - Epoch 10:   0%|          | 0/30 [00:15<?, ?it/s]


✅ LoRA model saved for Ram at /content/drive/MyDrive/lora_trained_model/Ram_lora

🚀 Training for character: Tina


Tina - Epoch 10:   0%|          | 0/30 [00:15<?, ?it/s]

✅ LoRA model saved for Tina at /content/drive/MyDrive/lora_trained_model/Tina_lora
🎉 Training Completed for All Characters!


In [ ]:
import torch
import os
import json
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# ✅ Load Model for Inference with LoRA for a specific character
def load_model_for_inference(base_model_path, lora_weights_path, character):
    """Loads the base Stable Diffusion model and applies LoRA weights for the specific character."""
    pipe = StableDiffusionPipeline.from_pretrained(base_model_path, torch_dtype=torch.float16)
    pipe.to("cuda")

    # ✅ Define the correct path for LoRA adapter based on character
    lora_adapter_path = os.path.join(lora_weights_path, f"{character}_lora", "lora_adapter")

    # ✅ Print the path for verification
    print(f"🔍 Checking LoRA adapter path: {lora_adapter_path}")

    # ✅ Check if the LoRA adapter exists and load it
    if os.path.exists(lora_adapter_path):
        print(f"✅ Loading LoRA adapter for {character} from {lora_adapter_path}...")
        pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_adapter_path)
        pipe.unet = pipe.unet.merge_and_unload()
        print(f"✅ LoRA successfully merged for {character}.")
    else:
        print(f"❌ LoRA adapter for {character} not found at {lora_adapter_path}.")

    return pipe

# ✅ Configuration
class Config:
    base_model = "stabilityai/stable-diffusion-2-1-base"  # Base model for Stable Diffusion
    lora_weights_path = "/content/drive/MyDrive/lora_trained_model"  # Correct path where LoRA weights are saved
    output_dir = "./panels/"  # Folder to save the generated comic panels
    json_file = "hindi_story.json"  # Input JSON file for comic generation

# ✅ Ensure output directory exists
os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Generate image for each scene based on the character-specific model
def generate_comic_from_json(json_file):
    """Reads a JSON file and generates comic panels for each scene based on character-specific models."""

    # ✅ Load JSON data
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    scenes = data["scenes"]
    location = ""  # Keep track of the location

    # ✅ Generate panels for each scene
    for scene in scenes:
        scene_number = scene["scene_number"]
        scene_text = scene["scene_text"]

        # ✅ Check if dialogues exist and are in the correct format
        if scene.get("dialogues"):
            if isinstance(scene["dialogues"], list):
                # If dialogues are a list of strings, use the first one for dialogue_text
                dialogue_text = scene["dialogues"][0] if scene["dialogues"] else "No dialogue found"
                character = "default_character"  # Use a default character if not available
            else:
                # If dialogues are in a dictionary format, extract character and text
                character = scene["dialogues"].get("character", "default_character")
                dialogue_text = scene["dialogues"].get("text", "No dialogue found")
        else:
            # If no dialogues found, use a default character and dialogue
            character = "default_character"
            dialogue_text = "No dialogue found"

        # ✅ If location is not specified, use the last known location
        if 'location' in scene and scene['location']:
            location = scene['location']

        if location:
            # Add location to scene_text if not already part of it
            if location.lower() not in scene_text.lower():
                scene_text = f"In the {location}, {scene_text}"

        # ✅ Load the character-specific model for inference
        inference_pipe = load_model_for_inference(Config.base_model, Config.lora_weights_path, character)

        # ✅ Add comic-style details to the prompt using scene_text and dialogue_text
        prompt = (
            f"{scene_text}, with dialogue: '{dialogue_text}', highly detailed comic-style illustration, "
            "vibrant colors, inked outlines, dynamic composition, expressive characters, in the style of graphic novels."
        )

        print(f"🎨 Generating panel for Scene {scene_number} with character {character}: {prompt}")

        # ✅ Clear VRAM before generating
        torch.cuda.empty_cache()

        # ✅ Generate image
        with torch.autocast("cuda"):
            image = inference_pipe(prompt).images[0]

        # ✅ Save the image in BMP format
        output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}_{character}.bmp")
        image.save(output_path)
        print(f"✅ Panel {scene_number} for {character} saved: {output_path}")

    print("🎉 All comic panels generated successfully!")

# ✅ Run the script
if __name__ == "__main__":
    generate_comic_from_json(Config.json_file)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (114 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["।, with dialogue : ' no dialogue found ', highly detailed comic - style illustration , vibrant colors , inked outlines , dynamic composition , expressive characters , in the style of graphic novels ."]


🔍 Checking LoRA adapter path: /content/drive/MyDrive/lora_trained_model/default_character_lora/lora_adapter
❌ LoRA adapter for default_character not found at /content/drive/MyDrive/lora_trained_model/default_character_lora/lora_adapter.
🎨 Generating panel for Scene 1 with character default_character: राम, एक सुंदर लड़का, जंगल में अपने बात करने वाले कुत्ते कालू के साथ चल रहा है।, with dialogue: 'No dialogue found', highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, in the style of graphic novels.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Panel 1 for default_character saved: ./panels/comic_panel_1_default_character.bmp


...

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['outlines , dynamic composition , expressive characters , in the style of graphic novels .']


🔍 Checking LoRA adapter path: /content/drive/MyDrive/lora_trained_model/default_character_lora/lora_adapter
❌ LoRA adapter for default_character not found at /content/drive/MyDrive/lora_trained_model/default_character_lora/lora_adapter.
🎨 Generating panel for Scene 5 with character default_character: बाद में तीनों एक झील के पास बैठते हैं और बातें करते हैं।, with dialogue: 'No dialogue found', highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, in the style of graphic novels.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Panel 5 for default_character saved: ./panels/comic_panel_5_default_character.bmp
🎉 All comic panels generated successfully!


In [ ]:
import torch
import os
import json
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# ✅ Load Model for Inference with LoRA for a specific character
def load_model_for_inference(base_model_path, lora_weights_path, character):
    """Loads the base Stable Diffusion model and applies LoRA weights for the specific character."""
    pipe = StableDiffusionPipeline.from_pretrained(base_model_path, torch_dtype=torch.float16)
    pipe.to("cuda")

    # ✅ Define the correct path for LoRA adapter based on character
    lora_adapter_path = os.path.join(lora_weights_path, f"{character}_lora", "lora_adapter")

    # ✅ Print the path for verification
    print(f"🔍 Checking LoRA adapter path: {lora_adapter_path}")

    # ✅ Check if the LoRA adapter exists and load it
    if os.path.exists(lora_adapter_path):
        print(f"✅ Loading LoRA adapter for {character} from {lora_adapter_path}...")
        pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_adapter_path)
        pipe.unet = pipe.unet.merge_and_unload()
        print(f"✅ LoRA successfully merged for {character}.")
    else:
        print(f"❌ LoRA adapter for {character} not found at {lora_adapter_path}.")

    return pipe

# ✅ Configuration
class Config:
    base_model = "stabilityai/stable-diffusion-2-1-base"  # Base model for Stable Diffusion
    lora_weights_path = "/content/drive/MyDrive/lora_trained_model"  # Correct path where LoRA weights are saved
    output_dir = "./panels/"  # Folder to save the generated comic panels
    json_file = "hindi_story.json"  # Input JSON file for comic generation

# ✅ Ensure output directory exists
os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Generate image for each scene based on the character-specific model
def generate_comic_from_json(json_file):
    """Reads a JSON file and generates comic panels for each scene based on character-specific models."""

    # ✅ Load JSON data
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    scenes = data["scenes"]
    location = ""  # Keep track of the location

    # ✅ Define Hindi-to-English character mapping for LoRA
    character_map = {
        "राम": "Ram",
        "कालू": "Bruno",
        "सोनी": "Tina"
    }

    # ✅ Generate panels for each scene
    for scene in scenes:
        scene_number = scene["scene_number"]
        scene_text = scene["scene_text"]

        # ✅ Check if dialogues exist, and get the character if dialogues are present
        if scene["dialogues"]:
            # If dialogue is just a list, assume one character is speaking
            if isinstance(scene["dialogues"][0], dict) and "character" in scene["dialogues"][0]:
                character = scene["dialogues"][0]["character"]
            else:
                character = scene["characters_in_scene"][0]
        else:
            character = scene["characters_in_scene"][0] if scene["characters_in_scene"] else "default_character"
            print(f"⚠️ No dialogues found. Using default character: {character}")

        # ✅ If location is not specified, use the last known location
        if 'location' in scene and scene['location']:
            location = scene['location']

        if location:
            # Add location to scene_text if not already part of it
            if location.lower() not in scene_text.lower():
                scene_text = f"In the {location}, {scene_text}"

        # ✅ Map Hindi character to English LoRA folder name
        mapped_character = character_map.get(character, character)

        # ✅ Load the character-specific model for inference
        inference_pipe = load_model_for_inference(Config.base_model, Config.lora_weights_path, mapped_character)

        # ✅ Add comic-style details to the prompt
        prompt = (
            f"{scene_text}, highly detailed comic-style illustration, vibrant colors, "
            "inked outlines, dynamic composition, expressive characters, in the style of graphic novels."
        )

        print(f"🎨 Generating panel for Scene {scene_number} with character {character}: {prompt}")

        # ✅ Clear VRAM before generating
        torch.cuda.empty_cache()

        # ✅ Generate image
        with torch.autocast("cuda"):
            image = inference_pipe(prompt).images[0]

        # ✅ Save the image in BMP format
        output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}_{character}.bmp")
        image.save(output_path)
        print(f"✅ Panel {scene_number} for {character} saved: {output_path}")

    print("🎉 All comic panels generated successfully!")

# ✅ Run the script
if __name__ == "__main__":
    generate_comic_from_json(Config.json_file)


⚠️ No dialogues found. Using default character: कालू


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

🔍 Checking LoRA adapter path: /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter
✅ Loading LoRA adapter for Bruno from /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter...


Token indices sequence length is longer than the specified maximum sequence length for this model (106 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['।, highly detailed comic - style illustration , vibrant colors , inked outlines , dynamic composition , expressive characters , in the style of graphic novels .']


✅ LoRA successfully merged for Bruno.
🎨 Generating panel for Scene 1 with character कालू: राम, एक सुंदर लड़का, जंगल में अपने बात करने वाले कुत्ते कालू के साथ चल रहा है।, highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, in the style of graphic novels.


...

🔍 Checking LoRA adapter path: /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter
✅ Loading LoRA adapter for Bruno from /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter...


Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['in the style of graphic novels .']


✅ LoRA successfully merged for Bruno.
🎨 Generating panel for Scene 5 with character कालू: बाद में तीनों एक झील के पास बैठते हैं और बातें करते हैं।, highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, in the style of graphic novels.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Panel 5 for कालू saved: ./panels/comic_panel_5_कालू.bmp
🎉 All comic panels generated successfully!


In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=423005307c00530b3f030d17fece9f498704abd820a9504cc2b8d88ef1ae625f
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [ ]:
import json
from googletrans import Translator

def translate_story(input_file, output_file):
    # Initialize translator
    translator = Translator()

    # Load Hindi story
    with open(input_file, 'r', encoding='utf-8') as f:
        hindi_data = json.load(f)

    # Translate the main story text
    translated_story = translator.translate(hindi_data['original_hindi'], src='hi', dest='en').text

    # Prepare English data structure
    english_data = {
        "original_english": translated_story,
        "characters": [],
        "scenes": []
    }

    # Translate character names (we'll keep these as is for LoRA mapping)
    english_data['characters'] = hindi_data['characters']

    # Translate each scene
    for scene in hindi_data['scenes']:
        translated_scene = {
            "scene_number": scene["scene_number"],
            "location": translator.translate(scene["location"], src='hi', dest='en').text,
            "scene_text": translator.translate(scene["scene_text"], src='hi', dest='en').text,
            "characters_in_scene": scene["characters_in_scene"],  # Keep original for mapping
            "emotion": scene["emotion"],  # Already in English
            "position": scene["position"],  # Already in English
            "dialogues": [translator.translate(d, src='hi', dest='en').text for d in scene["dialogues"]]
        }
        english_data['scenes'].append(translated_scene)

    # Save translated story
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(english_data, f, ensure_ascii=False, indent=4)

    print(f"✅ Successfully translated {input_file} to {output_file}")

# Usage
translate_story("hindi_story.json", "translated_story.json")

✅ Successfully translated hindi_story.json to translated_story.json


In [ ]:
import json

# Your Hindi to English character mapping
character_map = {
    "कालू": "Bruno",
    "सोनी": "Tina",
    "राम": "Ram"
}

# Optional: LoRA adapter mapping (if needed later)
lora_model_map = {
    "Kalu": "Bruno",
    "Soni": "Tina",
    "Ram": "Ram"
}

# Load the JSON file
with open("translated_story.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Update character names in each scene
for scene in data["scenes"]:
    scene["characters_in_scene"] = [character_map.get(c, c) for c in scene["characters_in_scene"]]

# Replace characters list with English-mapped ones (if needed)
data["characters"] = [character_map.get(c, c) for c in data["characters"]]

# Save the updated file
with open("english.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print("✅ Character mapping done and saved to 'english.json'")


✅ Character mapping done and saved to 'english.json'


In [ ]:
'''import torch
import os
import json
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# ✅ Configuration
class Config:
    base_model = "stabilityai/stable-diffusion-2-1-base"
    lora_weights_path = "/content/drive/MyDrive/lora_trained_model"
    output_dir = "./panels/"
    json_file = "english.json"

os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Cache to avoid reloading models
pipe_cache = {}

def load_model_for_inference(character):
    """Load and cache model for a specific character using LoRA."""
    if character in pipe_cache:
        return pipe_cache[character]

    print(f"🧠 Loading model for {character}")
    pipe = StableDiffusionPipeline.from_pretrained(Config.base_model, torch_dtype=torch.float16).to("cuda")

    lora_path = os.path.join(Config.lora_weights_path, f"{character}_lora", "lora_adapter")
    if os.path.exists(lora_path):
        print(f"🔗 Applying LoRA for {character} from {lora_path}")
        pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_path).merge_and_unload()
    else:
        print(f"⚠️ No LoRA found for {character}, using base model.")

    pipe_cache[character] = pipe
    return pipe

def generate_comic_from_json(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    last_location = None

    for scene in data["scenes"]:
        scene_number = scene["scene_number"]
        scene_text = scene["scene_text"]
        location = scene.get("location") or last_location or "Unknown Place"
        last_location = location

        # Choose character
        if scene["dialogues"]:
            character = scene["characters_in_scene"][0] if scene["characters_in_scene"] else data["characters"][0]
        else:
            character = scene["characters_in_scene"][0] if scene["characters_in_scene"] else data["characters"][0]

        # Build prompt
        if location.lower() not in scene_text.lower():
            scene_text = f"In the {location}, {scene_text}"
        prompt = (
            f"{scene_text}, highly detailed comic-style illustration, vibrant colors, "
            "inked outlines, dynamic composition, expressive characters, graphic novel style."
        )

        # Load model
        try:
            torch.cuda.empty_cache()
            pipe = load_model_for_inference(character)

            print(f"🎨 Scene {scene_number}: Prompt -> {prompt}")
            with torch.autocast("cuda"):
                image = pipe(prompt).images[0]

            output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}_{character}.bmp")
            image.save(output_path)
            print(f"✅ Saved panel at {output_path}")

        except Exception as e:
            print(f"❌ Error generating scene {scene_number}: {e}")

    print("✅✅ Comic generation complete!")

# ✅ Run
if __name__ == "__main__":
    generate_comic_from_json(Config.json_file)'''


🧠 Loading model for Bruno


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

🔗 Applying LoRA for Bruno from /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter
🎨 Scene 1: Prompt -> Ram, a beautiful boy, is walking with Kalu, a dog in the forest., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_1_Bruno.bmp
🧠 Loading model for Tina


...

✅ Saved panel at ./panels/comic_panel_3_Bruno.bmp
🎨 Scene 4: Prompt -> In the Forest, Soni says 'Hi'., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_4_Tina.bmp
🎨 Scene 5: Prompt -> Later the three sit near a lake and talk., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_5_Bruno.bmp
✅✅ Comic generation complete!


In [ ]:
'''import torch
import os
import json
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# ✅ Configuration
class Config:
    base_model = "stabilityai/stable-diffusion-2-1-base"
    lora_weights_path = "/content/drive/MyDrive/lora_trained_model"
    output_dir = "./panels/"
    json_file = "english.json"

os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Character descriptions (used in prompt for better generation)
character_descriptions = {
    "Bruno": "a playful brown dog with a wagging tail",
    "Tina": "a cheerful girl with braided hair and bright eyes",
    "Ram": "a handsome boy with traditional clothes and a gentle smile"
}

# ✅ Cache for model pipes
pipe_cache = {}

def load_model_for_inference(character):
    """Load and cache model for a specific character using LoRA."""
    if character in pipe_cache:
        return pipe_cache[character]

    print(f"🧠 Loading model for {character}")
    pipe = StableDiffusionPipeline.from_pretrained(Config.base_model, torch_dtype=torch.float16).to("cuda")

    lora_path = os.path.join(Config.lora_weights_path, f"{character}_lora", "lora_adapter")
    if os.path.exists(lora_path):
        print(f"🔗 Applying LoRA for {character} from {lora_path}")
        pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_path).merge_and_unload()
    else:
        print(f"⚠️ No LoRA found for {character}, using base model.")

    pipe_cache[character] = pipe
    return pipe

def generate_comic_from_json(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    last_location = None

    for scene in data["scenes"]:
        scene_number = scene["scene_number"]
        scene_text = scene["scene_text"]
        location = scene.get("location") or last_location or "Unknown Place"
        last_location = location

        # Choose a character (dialogue character or first visible one)
        if scene["dialogues"]:
            character = scene["characters_in_scene"][0] if scene["characters_in_scene"] else data["characters"][0]
        else:
            character = scene["characters_in_scene"][0] if scene["characters_in_scene"] else data["characters"][0]

        # ✅ Get character description
        char_desc = character_descriptions.get(character, character)

        # ✅ Update scene text with character description and location if needed
        if location.lower() not in scene_text.lower():
            scene_text = f"In the {location}, {char_desc} is present. {scene_text}"

        # ✅ Final prompt
        prompt = (
            f"{scene_text}, highly detailed comic-style illustration, vibrant colors, "
            "inked outlines, dynamic composition, expressive characters, graphic novel style."
        )

        # ✅ Load model and generate image
        try:
            torch.cuda.empty_cache()
            pipe = load_model_for_inference(character)

            print(f"🎨 Scene {scene_number}: Prompt -> {prompt}")
            with torch.autocast("cuda"):
                image = pipe(prompt).images[0]

            output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}_{character}.bmp")
            image.save(output_path)
            print(f"✅ Saved panel at {output_path}")

        except Exception as e:
            print(f"❌ Error generating scene {scene_number}: {e}")

    print("✅✅ Comic generation complete!")

# ✅ Run
if __name__ == "__main__":
    generate_comic_from_json(Config.json_file)'''


🧠 Loading model for Bruno


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

🔗 Applying LoRA for Bruno from /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter
🎨 Scene 1: Prompt -> Ram, a beautiful boy, is walking with Kalu, a dog in the forest., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_1_Bruno.bmp
🧠 Loading model for Tina


...

✅ Saved panel at ./panels/comic_panel_3_Bruno.bmp
🎨 Scene 4: Prompt -> In the Forest, a cheerful girl with braided hair and bright eyes is present. Soni says 'Hi'., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_4_Tina.bmp
🎨 Scene 5: Prompt -> Later the three sit near a lake and talk., highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_5_Bruno.bmp
✅✅ Comic generation complete!


In [ ]:
import torch
import os
import json
from diffusers import StableDiffusionPipeline
from peft import PeftModel

# ✅ Configuration
class Config:
    base_model = "stabilityai/stable-diffusion-2-1-base"
    lora_weights_path = "/content/drive/MyDrive/lora_trained_model"
    output_dir = "./panels/"
    json_file = "english.json"

# ✅ Create output folder
os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Style prompts based on known visual references
character_prompts = {
    "Ram": "a charming young man with a blonde hair and blue eyes, inspired by Flynn Rider from Tangled, Disney-style",
    "Bruno": "a goofy, friendly Great Dane dog, specifically a cartoon dog like Scooby-Doo",
    "Tina": "a brave island girl with long curly black hair and tropical attire, inspired by Moana, Disney-style"
}

# ✅ Cache for reusing loaded models
pipe_cache = {}

def load_model_for_inference(character):
    """Load and cache model for a specific character using LoRA."""
    if character in pipe_cache:
        return pipe_cache[character]

    print(f"🧠 Loading model for {character}")
    pipe = StableDiffusionPipeline.from_pretrained(
        Config.base_model, torch_dtype=torch.float16
    ).to("cuda")

    lora_path = os.path.join(Config.lora_weights_path, f"{character}_lora", "lora_adapter")
    if os.path.exists(lora_path):
        print(f"🔗 Applying LoRA for {character} from {lora_path}")
        pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_path).merge_and_unload()
    else:
        print(f"⚠️ No LoRA found for {character}, using base model.")

    pipe_cache[character] = pipe
    return pipe

def generate_comic_from_json(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    last_location = None

    for scene in data["scenes"]:
        scene_number = scene["scene_number"]
        scene_text = scene["scene_text"]
        location = scene.get("location") or last_location or "Unknown Place"
        last_location = location

        # Determine main character for the scene
        character = (
            scene["characters_in_scene"][0]
            if scene["characters_in_scene"]
            else data["characters"][0]
        )

        # Add location context if needed
        if location.lower() not in scene_text.lower():
            scene_text = f"In the {location}, {scene_text}"

        # Add style description
        style_desc = character_prompts.get(character, "")
        prompt = (
            f"{scene_text}. {style_desc}, highly detailed comic-style illustration, vibrant colors, "
            "inked outlines, dynamic composition, expressive characters, graphic novel style."
        )

        # Load LoRA model
        try:
            torch.cuda.empty_cache()
            pipe = load_model_for_inference(character)

            print(f"🎨 Scene {scene_number}: Prompt -> {prompt}")
            with torch.autocast("cuda"):
                image = pipe(prompt).images[0]

            output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}.bmp")
            image.save(output_path)
            print(f"✅ Saved panel at {output_path}")

        except Exception as e:
            print(f"❌ Error generating scene {scene_number}: {e}")

    print("✅✅ Comic generation complete!")

# ✅ Run the script
if __name__ == "__main__":
    generate_comic_from_json(Config.json_file)


🧠 Loading model for Bruno


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

🔗 Applying LoRA for Bruno from /content/drive/MyDrive/lora_trained_model/Bruno_lora/lora_adapter
🎨 Scene 1: Prompt -> Ram, a beautiful boy, is walking with Kalu, a dog in the forest.. a goofy, friendly Great Dane dog, specifically a cartoon dog like Scooby-Doo, highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_1_Bruno.bmp
🧠 Loading model for Tina


...

✅ Saved panel at ./panels/comic_panel_3_Bruno.bmp
🎨 Scene 4: Prompt -> In the Forest, Soni says 'Hi'.. a brave island girl with long curly black hair and tropical attire, inspired by Moana, Disney-style, highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_4_Tina.bmp
🎨 Scene 5: Prompt -> Later the three sit near a lake and talk.. a goofy, friendly Great Dane dog, specifically a cartoon dog like Scooby-Doo, highly detailed comic-style illustration, vibrant colors, inked outlines, dynamic composition, expressive characters, graphic novel style.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved panel at ./panels/comic_panel_5_Bruno.bmp
✅✅ Comic generation complete!


In [ ]:
# Install Noto Sans Devanagari
!apt-get install -y fonts-noto


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-noto-cjk fonts-noto-cjk-extra fonts-noto-color-emoji fonts-noto-core
  fonts-noto-extra fonts-noto-mono fonts-noto-ui-core fonts-noto-ui-extra
  fonts-noto-unhinted
The following NEW packages will be installed:
  fonts-noto fonts-noto-cjk fonts-noto-cjk-extra fonts-noto-color-emoji
  fonts-noto-core fonts-noto-extra fonts-noto-mono fonts-noto-ui-core
  fonts-noto-ui-extra fonts-noto-unhinted
0 upgraded, 10 newly installed, 0 to remove and 34 not upgraded.
Need to get 317 MB of archives.
After this operation, 790 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-core all 20201225-1build1 [12.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-noto all 20201225-1build1 [16.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-cjk all 1:

In [ ]:
'''import json
import os
from PIL import Image, ImageDraw, ImageFont

# ✅ Configuration
class Config:
    json_path = "hindi_story.json"
    panels_dir = "/content/panels"
    font_path = "/usr/share/fonts/truetype/noto/NotoSansDevanagari-Regular.ttf"
    font_size = 36
    output_dir = "/content/panels_with_dialogue"

os.makedirs(Config.output_dir, exist_ok=True)

# ✅ Load font
try:
    font = ImageFont.truetype(Config.font_path, Config.font_size)
except OSError:
    raise RuntimeError(f"Could not load font from: {Config.font_path}")

# ✅ Load story JSON
with open(Config.json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Add dialogues to each panel with background boxes
for scene in data["scenes"]:
    scene_number = scene["scene_number"]
    dialogues = scene.get("dialogues", [])

    if not dialogues:
        continue

    image_path = os.path.join(Config.panels_dir, f"comic_panel_{scene_number}.bmp")
    if not os.path.exists(image_path):
        print(f"⚠️ Scene {scene_number}: No matching image found.")
        continue

    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Starting position
    x, y = 50, 50
    line_height = Config.font_size + 30
    padding = 15

    for dialogue in dialogues:
        # Get text bounding box
        bbox = draw.textbbox((x, y), dialogue, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Draw white rectangle background with black border
        box_coords = [x - padding, y - padding, x + text_width + padding, y + text_height + padding]
        draw.rectangle(box_coords, fill="white", outline="black", width=2)

        # Draw the dialogue text
        draw.text((x, y), dialogue, font=font, fill="black")

        # Move to next line
        y += line_height

    output_path = os.path.join(Config.output_dir, f"comic_panel_{scene_number}.bmp")
    image.save(output_path)
    print(f"💬 Scene {scene_number}: Dialogue box added -> Saved to {output_path}")

print("🎉 All dialogues added with background boxes!")'''


💬 Scene 3: Dialogue box added -> Saved to /content/panels_with_dialogue/comic_panel_3.bmp
💬 Scene 4: Dialogue box added -> Saved to /content/panels_with_dialogue/comic_panel_4.bmp
🎉 All dialogues added with background boxes!


In [ ]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

# ✅ Configuration
class Config:
    json_path = "hindi_story.json"
    panels_dir = "/content/panels"
    font_path = "/usr/share/fonts/truetype/noto/NotoSansDevanagari-Regular.ttf"
    font_size = 36
    output_dir_with_dialogue = "/content/panels_with_dialogue"
    output_dir_without_dialogue = "/content/panels_without_dialogue"

# ✅ Create output directories
os.makedirs(Config.output_dir_with_dialogue, exist_ok=True)
os.makedirs(Config.output_dir_without_dialogue, exist_ok=True)

# ✅ Load font
try:
    font = ImageFont.truetype(Config.font_path, Config.font_size)
except OSError:
    raise RuntimeError(f"Could not load font from: {Config.font_path}")

# ✅ Load story JSON
with open(Config.json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Process each scene
for scene in data["scenes"]:
    scene_number = scene["scene_number"]
    dialogues = scene.get("dialogues", [])

    image_path = os.path.join(Config.panels_dir, f"comic_panel_{scene_number}.bmp")
    if not os.path.exists(image_path):
        print(f"⚠️ Scene {scene_number}: No matching image found.")
        continue

    # ✅ Save the original image as "without dialogue"
    image = Image.open(image_path)
    no_dialogue_path = os.path.join(Config.output_dir_without_dialogue, f"comic_panel_{scene_number}.bmp")
    image.save(no_dialogue_path)
    print(f"📁 Scene {scene_number}: Saved without dialogue to -> {no_dialogue_path}")

    # ✅ If no dialogues, skip creating a second version
    if not dialogues:
        continue

    # ✅ Create copy for dialogue version
    image_with_dialogue = image.copy()
    draw = ImageDraw.Draw(image_with_dialogue)

    x, y = 50, 50
    line_height = Config.font_size + 30
    padding = 15

    for dialogue in dialogues:
        bbox = draw.textbbox((x, y), dialogue, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # White bubble with black outline
        box_coords = [x - padding, y - padding, x + text_width + padding, y + text_height + padding]
        draw.rectangle(box_coords, fill="white", outline="black", width=2)

        draw.text((x, y), dialogue, font=font, fill="black")
        y += line_height

    # ✅ Save dialogue image
    dialogue_path = os.path.join(Config.output_dir_with_dialogue, f"comic_panel_{scene_number}.bmp")
    image_with_dialogue.save(dialogue_path)
    print(f"💬 Scene {scene_number}: Dialogue version saved to -> {dialogue_path}")

print("✅✅ Done! All panels saved with and without dialogues.")


📁 Scene 1: Saved without dialogue to -> /content/panels_without_dialogue/comic_panel_1.bmp
📁 Scene 2: Saved without dialogue to -> /content/panels_without_dialogue/comic_panel_2.bmp
📁 Scene 3: Saved without dialogue to -> /content/panels_without_dialogue/comic_panel_3.bmp
💬 Scene 3: Dialogue version saved to -> /content/panels_with_dialogue/comic_panel_3.bmp
📁 Scene 4: Saved without dialogue to -> /content/panels_without_dialogue/comic_panel_4.bmp
💬 Scene 4: Dialogue version saved to -> /content/panels_with_dialogue/comic_panel_4.bmp
📁 Scene 5: Saved without dialogue to -> /content/panels_without_dialogue/comic_panel_5.bmp
✅✅ Done! All panels saved with and without dialogues.


In [ ]:
from PIL import Image
import os
import json

# ✅ Configuration
class Config:
    json_path = "hindi_story.json"
    dialogue_dir = "/content/panels_with_dialogue"
    no_dialogue_dir = "/content/panels_without_dialogue"
    final_strip_path = "/content/final_comic_strip.bmp"
    layout = "horizontal"  # or "vertical"

# ✅ Load JSON
with open(Config.json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ Gather all scene images in order
scene_images = []
for scene in data["scenes"]:
    scene_number = scene["scene_number"]
    has_dialogue = bool(scene.get("dialogues"))

    # Pick correct image based on presence of dialogues
    if has_dialogue:
        img_path = os.path.join(Config.dialogue_dir, f"comic_panel_{scene_number}.bmp")
    else:
        img_path = os.path.join(Config.no_dialogue_dir, f"comic_panel_{scene_number}.bmp")

    if not os.path.exists(img_path):
        print(f"⚠️ Image for scene {scene_number} not found at {img_path}, skipping.")
        continue

    image = Image.open(img_path)
    scene_images.append(image)

# ✅ Combine images
if not scene_images:
    raise RuntimeError("No images found to create comic strip.")

# Dimensions for final canvas
if Config.layout == "horizontal":
    total_width = sum(img.width for img in scene_images)
    max_height = max(img.height for img in scene_images)
    final_image = Image.new("RGB", (total_width, max_height), color=(255, 255, 255))

    x_offset = 0
    for img in scene_images:
        final_image.paste(img, (x_offset, 0))
        x_offset += img.width

else:  # vertical layout
    max_width = max(img.width for img in scene_images)
    total_height = sum(img.height for img in scene_images)
    final_image = Image.new("RGB", (max_width, total_height), color=(255, 255, 255))

    y_offset = 0
    for img in scene_images:
        final_image.paste(img, (0, y_offset))
        y_offset += img.height

# ✅ Save the final comic strip
final_image.save(Config.final_strip_path)
print(f"🎉 Comic strip created: {Config.final_strip_path}")


🎉 Comic strip created: /content/final_comic_strip.bmp
